### IMPORTS

In [1]:
import numpy as np

In [2]:
from sklearn.linear_model import LinearRegression

In [3]:
import tensorflow as tf
from tensorflow.python.keras import Sequential, Input, Model
from tensorflow.python.keras.layers import Dense, LSTM, Dropout

In [4]:
import ipynb.fs.full.splitting as splitting

In [5]:
# from tcn import TCN, tcn_full_summary

### LINEAR REGRESSION

In [6]:
def linear_regression(data, settings):
    
    # INSTANTIATE MODEL
    linear = LinearRegression()
    
    # TRAIN WITH TRAIN DATA
    model = linear.fit(data['train']['features'], data['train']['labels'])
    
    # PREDICT USING TEST DATA
    predictions = model.predict(data['test']['features'])
    
    return np.ndarray.flatten(predictions)

### LONG-SHORT TERM MEMORY

In [89]:
def add_lstm_layer(name, settings, model, index, shape):
    
    # AVAILABLE LSTM LAYERS
    available = {
        'lstm': LSTM,
        'dropout': Dropout,
        'dense': Dense
    }

    # SELECT THE CORRECT FUNCTION
    func = available[name]

    # INJECT INPUT LAYER DIMENSIONS TO SETTINGS
    if index == 0:
        settings['input_shape'] = (shape[1], shape[2])
        model.add(func(**settings))

    # OTHERWISE, DEFAULT TO BASE SETTINGS
    else:
        model.add(func(**settings))

In [88]:
def long_short_term(data, settings):
    
    # INSTANTIATE MODEL
    model = Sequential()
    
    # TRAIN DATA GENERATOR
    train_generator = splitting.generator(
        data['train'],
        settings['morph'],
        shuffle=True
    )
    
    # LOOP THROUGH REQUESTED MODEL LAYERS
    for index, layer in enumerate(settings['layers']):
        
        # LAYER PROPS
        name = list(layer)[0]
        params = layer[name]
        
        # ADD LAYER
        add_lstm_layer(
            name,
            params,
            model,
            index,
            train_generator[0][0].shape
        )
    
    # COMPILE THE MODEL
    model.compile(
        loss=settings['loss'],
        optimizer=settings['optimizer']
    )
    
    # FIT USING THE TRAIN GENERATOR
    model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=settings['epochs'],
        verbose=0
    )
    
    # TEST DATA GENERATOR
    test_generator = splitting.generator(
        data['test'],
        settings['morph'],
        shuffle=False
    )
    
    # PREDICT USING TEST DATA
    predictions = model.predict(test_generator)
    
    return np.ndarray.flatten(predictions)

### TEMPORAL CONVOLUTIONAL NETWORK

### START TRAINING A MODEL

In [ ]:
def start(dataset, name, settings):
    
    # AVAILABLE MODELS
    available = {
        'linreg': linear_regression,
        'lstm': long_short_term
    }
    
    # SELECT THE CORRECT FUNCTION & START
    return available[name](dataset, settings)

### TESTING

In [67]:
test = {
    'x': 'foobar'
}

In [68]:
def foobar(**kwargs):
    for key, value in kwargs.items():
        print ("%s == %s" %(key, value))

In [69]:
foobar(**test)

x == foobar
